In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


**Seperate data into training and test set**

In [46]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print('Train Dataset shape: {a}'.format(a = train_df.shape))
print('Test Dataset shape: {a}'.format(a = test_df.shape))
train_df.head()

Train Dataset shape: (42000, 785)
Test Dataset shape: (28000, 784)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
X_train = train_df.drop('label', axis=1).to_numpy()
y_train = train_df['label'].to_numpy()
print('X_train shape: {a}'.format(a = X_train.shape))
print('y_train shape: {a}'.format(a = y_train.shape))
X_test = test_df.to_numpy()

X_train shape: (42000, 784)
y_train shape: (42000,)


**Multiclass Neural network using tensorflow**

- Softmax
- Adam optimizer

In [28]:
tf.random.set_seed(776)
model = Sequential([
    tf.keras.Input(shape=(784,)),
    Dense(40, activation='relu', name = "Layer1"),
    Dense(20, activation='relu',  name = "Layer2"),
    Dense(10, activation='linear', name = "Layer3"), 
    ], name = "MNIST_Model" 
)

model.summary()

Model: "MNIST_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 40)                31400     
                                                                 
 Layer2 (Dense)              (None, 20)                820       
                                                                 
 Layer3 (Dense)              (None, 10)                210       
                                                                 
Total params: 32430 (126.68 KB)
Trainable params: 32430 (126.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

model.fit(X_train, y_train, epochs=40)

Epoch 1/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0726
Epoch 2/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0676
Epoch 3/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0678
Epoch 4/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0680
Epoch 5/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0632
Epoch 6/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0686
Epoch 7/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0601
Epoch 8/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0657
Epoch 9/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0578
Epoch 10/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0635
Epoch 11/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0610
Epoch 12/40
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0655
E

**Obtaining and comparing prediction results to actual results**

In [70]:
y_pred = np.zeros(y_train.shape[0])
prediction = model.predict(X_train)
    
for i in range(prediction.shape[0]):
    yhat = np.argmax(prediction[i])
    y_pred[i] = yhat
y_pred = y_pred.astype(int)
accuracy_score(y_train, y_pred)

1313/1313 [==============================] - 3s 2ms/step


0.9897857142857143

In [71]:
test_pred = np.zeros(X_test.shape[0])
prediction = model.predict(X_test)

for i in range(prediction.shape[0]):
    yhat = np.argmax(prediction[i])
    test_pred[i] = yhat


875/875 [==============================] - 2s 2ms/step


In [81]:
test_pred = test_pred.astype(int)
ImageId = np.arange(1, test_pred.shape[0] + 1)
output = pd.DataFrame({'ImageId':ImageId, 'Label':test_pred})

output.to_csv('submission.csv', index = False)
